### 사전 준비 사항 

#### (1) uv add (터미널)

```bash
uv add pdfplumber sentence-transformers faiss-cpu numpy torch python-dotenv openai
```

#### (2) .env 파일 세팅
```bash
OPENAI_API_KEY = ""
HF_TOKEN = ""
```

#### (3) pdf 파일 세팅
pdf 파일 100개를 `data/raw/files` 에 위치합니다.  
성능 개선을 위한 테스트에서는 전체를 대상으로 인덱싱하지 않고, 5개 인덱싱 코드 실행 권장(주석 해제)

In [2]:
import preprocess_v4 as pp
from preprocess_v4 import ALL_DATA

In [3]:
import pdfplumber
import os
from pathlib import Path
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import torch

/Users/won/dev/00_codeit/0_mission/200_DL_RAG/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
client = OpenAI()
MODEL = "gpt-4o-mini"

In [5]:
BASE_DIR = Path.cwd().parent  # /codeit-part3-team4
RAW_FOLDER = BASE_DIR / "data/raw/files"  # 수정된 경로

# PDF to text
def extract_text(pdf_path: Path | str) -> str:
    texts = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            texts.append(page.extract_text() or "")
    return "\n".join(texts)

# 폴더에서 PDF 목록 가져오기
def get_pdf_paths(folder_path: Path | str) -> list[Path]:
    folder = Path(folder_path)
    pdf_paths = [p for p in folder.glob("*.pdf")]
    return sorted(pdf_paths)

In [6]:
# Chunking
def chunk(text: str, size: int = 800) -> list[str]:
    return [text[i:i+size] for i in range(0, len(text), size)]

In [7]:
# 임베딩 및 인덱스 만들기
# 한국어 임베딩 모델
embed_model_name = "nlpai-lab/KoE5"
embed_model = SentenceTransformer(embed_model_name)

def build_index(chunks: list[str]):
    embs = embed_model.encode(chunks, convert_to_numpy=True, show_progress_bar=False)
    index = faiss.IndexFlatL2(embs.shape[1])
    index.add(embs.astype("float32"))
    return index, chunks

Loading weights: 100%|██████████| 391/391 [00:00<00:00, 2236.42it/s, Materializing param=pooler.dense.weight]                               


In [8]:
# 질문 리스트 (key, 질문)
queries = [
    ("project_name", "사업(용역)명은 무엇인가?"),
    ("agency", "발주 기관(수요기관)은 어디인가?"),
    ("purpose", "사업 목적(추진 배경)은 무엇인가?"),
    ("budget", "총 사업 예산(사업비)은 얼마인가?"),
    ("contract_type", "계약 방식(일반경쟁/제한경쟁/협상에 의한 계약 등)은 무엇인가?"),
    ("deadline", "입찰/제안서 제출 마감일시는 언제인가?"),
    ("duration", "사업 수행 기간은 얼마나 되는가?"),
    ("requirements_must", "필수 요구사항(기능/성능/보안 등)은 무엇인가?"),
    ("eval_items", "평가 항목(기술/가격 등) 구성은 어떻게 되는가?"),
    ("price_eval", "가격 평가 방식(최저가/협상 등)은 무엇인가?"),
    ("eligibility", "참가 자격 요건(면허/실적/인증/등급)은 무엇인가?"),
]

def build_query_prompt(queries):
    return "\n".join(
        f"{i}. {q}"
        for i, (_, q) in enumerate(queries, start=1)
    )

In [9]:
# RFP 분석용 프롬프트
RFP_PROMPT = """
너는 정부·공공기관 제안요청서(RFP)를 분석하는 전문가다.
아래 컨텍스트는 하나의 정부 RFP 문서에서 추출된 내용이다.

[분석 규칙]
- 추측 금지, 문서에 명시된 내용만 사용
- 문서에 없으면 반드시 "명시 없음"
- **출력은 질문 개수와 동일한 줄 수**
- **각 줄에는 답변 텍스트만 작성**
- 번호, 질문 문장, '답:', 기호, 설명을 절대 포함하지 말 것

[질문 목록]
{questions}

[컨텍스트]
{context}

질문 순서대로 답변만 한 줄씩 출력하라.
"""

In [10]:
# 쿼리 답변
def answer(index, embed_model, chunks, queries, top_k: int = 15) -> dict:
    # 1. 검색용 대표 질문
    search_query = "RFP 주요 사업 정보 요약"
    q_emb = embed_model.encode([search_query], convert_to_numpy=True)
    _, I = index.search(q_emb.astype("float32"), top_k)
    context = "\n\n".join(chunks[i] for i in I[0])
    questions_text = build_query_prompt(queries)
    prompt = RFP_PROMPT.format(
        context=context,
        questions=questions_text
    )
    # 2. OpenAI API 호출
    resp = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": "RFP 전문 분석가. 형식 엄수."},
            {"role": "user", "content": prompt},
        ],
        max_tokens=2000,
        temperature=0.0,
    )
    raw_answer = resp.choices[0].message.content.strip()
    # 3. key: value 매핑
    results = {}
    for (key, _), line in zip(queries, raw_answer.splitlines()):
        line = line.strip()
        if line:
            results[key] = line
        else:
            results[key] = "명시 없음"
    return results

In [11]:
# 데이터 폴더 설정
docs = get_pdf_paths(RAW_FOLDER)
print(f"발견된 PDF: {len(docs)}개")
for i, doc in enumerate(docs):
    print(f"{i}: {doc.name}")

발견된 PDF: 100개
0: (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf
1: (사)부산국제영화제_2024년 BIFF & ACFM 온라인서비스 재개발 및 행사지원시.pdf
2: (사）한국대학스포츠협의회_KUSF 체육특기자 경기기록 관리시스템 개발.pdf
3: (재)예술경영지원센터_통합 정보시스템 구축 사전 컨설팅.pdf
4: 2025 구미 아시아육상경기선수권대회 조직위원회_2025 구미아시아육상경.pdf
5: BioIN_의료기기산업 종합정보시스템(정보관리기관) 기능개선 사업(2차).pdf
6: KOICA 전자조달_[긴급] [지문] [국제] 우즈베키스탄 열린 의정활동 상하원 .pdf
7: 경기도 안양시_호계체육관 배드민턴장 및 탁구장 예약시스템 구축 용역.pdf
8: 경기도 평택시_2024년도 평택시 버스정보시스템(BIS) 구축사업.pdf
9: 경기도사회서비스원_2024년 통합사회정보시스템 운영지원.pdf
10: 경상북도 봉화군_봉화군 재난통합관리시스템 고도화 사업(협상)(긴급).pdf
11: 경희대학교_[입찰공고] 산학협력단 정보시스

In [12]:
# 전체 문서별 인덱스 저장
doc_indexes = {}
for doc_path in docs:
    print(f"처리 중: {doc_path.name}")
    chunks = pp.chunk_from_alldata(doc_path.name, ALL_DATA)

    if chunks is None:
        text = pp.clean_text(extract_text(doc_path))
        chunks = chunk(text)

    index, chunks_list = build_index(chunks)
    doc_indexes[doc_path] = (index, chunks_list)
print("모든 문서 인덱싱 완료")

처리 중: (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf
처리 중: (사)부산국제영화제_2024년 BIFF & ACFM 온라인서비스 재개발 및 행사지원시.pdf
처리 중: (사）한국대학스포츠협의회_KUSF 체육특기자 경기기록 관리시스템 개발.pdf
처리 중: (재)예술경영지원센터_통합 정보시스템 구축 사전 컨설팅.pdf
처리 중: 2025 구미 아시아육상경기선수권대회 조직위원회_2025 구미아시아육상경.pdf
처리 중: BioIN_의료기기산업 종합정보시스템(정보관리기관) 기능개선 사업(2차).pdf
처리 중: KOICA 전자조달_[긴급] [지문] [국제] 우즈베키스탄 열린 의정활동 상하원 .pdf
처리 중: 경기도 안양시_호계체육관 배드민턴장 및 탁구장 예약시스템 구축 용역.pdf
처리 중: 경기도 평택시_2024년도 평택시 버스정보시스템(BIS) 구축사업.pdf
처리 중: 경기도사회서비스원_2024년 통합사회정보시스템 운영지원.pdf
처리 중: 경상북도 봉화군_봉화군 재난통합관리시스템 고도화 사업(협상)(긴급).pdf
처리 중: 경희대학교_[입찰공고] 산하

In [13]:
# # 전체 중 5개 문서만 문서별 인덱스 저장
# doc_indexes = {}
# for doc_path in docs[:5]:
#     print(f"처리 중: {doc_path.name}")
#     text = extract_text(doc_path)
#     chunks = chunk(text)
#     index, chunks_list = build_index(chunks)
#     doc_indexes[doc_path] = (index, chunks_list)
# print("5개 문서 인덱싱 완료")

In [14]:
# 테스트: 단일 문서
if docs:
    test_doc = docs[0]
    print(f"\n=== {test_doc.name} 분석 ===")
    index, chunks = doc_indexes[test_doc]
    result = answer(index, embed_model, chunks, queries)
    for k, v in result.items():
        print(f"{k}: {v}")


=== (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf 분석 ===
project_name: 벤처확인종합관리시스템 기능 고도화 용역사업
agency: 명시 없음
purpose: 명시 없음
budget: 명시 없음
contract_type: 명시 없음
deadline: 2024년 7월 10일
duration: 계약일로부터 14일 이내에 제출
requirements_must: 명시 없음
eval_items: 개발장비 요구사항, 통제 및 리스크 계획
price_eval: 기술 평가와 가격 평가
eligibility: 협상에 의한 계약


In [15]:
# 테스트: 전체 문서(비용 주의-주석 해제 후 사용)

import json

results = {}
for doc in docs:
    print(f"\n=== {doc.name} 분석 ===")
    if doc not in doc_indexes:
        print(f"[SKIP] 인덱스 없음: {doc.name}")
        continue
    index, chunks = doc_indexes[doc]
    result = answer(
        index=index,
        embed_model=embed_model,
        chunks=chunks,
        queries=queries
    )
    results[doc.name] = result
    for k, v in result.items():
        print(f"{k}: {v}")

# JSON 저장
output_path = BASE_DIR / "rfp_api_answer_results_v2.4.json"
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)
print(f"\n전체 문서 분석 완료! 저장 위치: {output_path}")


=== (사)벤처기업협회_2024년 벤처확인종합관리시스템 기능 고도화 용역사업 .pdf 분석 ===
project_name: 벤처확인종합관리시스템 기능 고도화 용역사업
agency: 명시 없음
purpose: 명시 없음
budget: 명시 없음
contract_type: 명시 없음
deadline: 2024년 7월 10일
duration: 계약일로부터 14일 이내
requirements_must: 필수 요구사항 명시 없음
eval_items: 기술 및 가격 평가
price_eval: 협상에 의한 계약
eligibility: 대기업 및 중견기업은 입찰에 참여할 수 없음, 상호출자제한기업집단소속회사는 입찰에 참여할 수 없음

=== (사)부산국제영화제_2024년 BIFF & ACFM 온라인서비스 재개발 및 행사지원시.pdf 분석 ===
project_name: BIFF&ACFM 온라인서비스 재개발 및 행사지원시스템 공급 용역
agency: 부산국제영화제
purpose: 웹 표준, 웹 접근성 및 웹 UI에 대한 기능개선으로 사용자 중심의 웹서비스 제공
budget: 금243,000,000원
contract_type: 제한 경쟁입찰(지역 제한)
deadline: 2024. 7. 2(화) 16:00
duration: 계약체결일 ~ 2025. 05. 31.
requirements_must: 명시 없음
eval_items: 기술부문, 사업관리 부문, 지원 부문
price_eval: 명시 없음
eligibility: 지방자치단체를 당사자로 하는 계약에 관한 법률 시행령 제92조에 해당되지 않는 기업, 주된 영업소의 소재지가 부산광역시인 기업, 소프트웨어사업자 등록, 대기업 및 중견기업 소프트웨어 사업자는 참여 불가

=== (사）한국대학스포츠협의회_KUSF 체육특기자 경기기록 관리ᄉ